In [1]:
import json
import logging
import os

from bs4 import BeautifulSoup
import requests
import ricecooker

from engageny_chef import get_text, get_parsed_html_from_url, make_fully_qualified_url
from engageny_chef import ENGAGENY_CC_START_URL
from engageny_chef import DATA_DIR, TREES_DATA_DIR, CRAWLING_STAGE_OUTPUT
from engageny_chef import LOGGER
LOGGER.addHandler(logging.StreamHandler())  # needed for logging in to work in notebook

import pprint
pp = pprint.PrettyPrinter(indent=4, width=80)

In [2]:
# basic reconaissance ... 
doc = get_parsed_html_from_url(ENGAGENY_CC_START_URL)

dual_toc_div = doc.find('div', id='mini-panel-common_core_curriculum')
ELA_toc = dual_toc_div.find('div', class_='panel-col-first')
MATH_toc = dual_toc_div.find('div', class_='panel-col-last')
MATH_grades_lis = MATH_toc.find_all('li')
len(MATH_grades_lis)

Starting new HTTPS connection (1): www.engageny.org
https://www.engageny.org:443 "GET /common-core-curriculum HTTP/1.1" 200 12228
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.fr

14

In [3]:
for grade_li in MATH_grades_lis:
    grade_path = grade_li.find('a')['href']
    grade_url = make_fully_qualified_url(grade_path)
    print('Grade:', get_text(grade_li), 'grade_url =', grade_url)

Grade: Prekindergarten Mathematics grade_url = https://www.engageny.org/content/prekindergarten-mathematics
Grade: Kindergarten Mathematics grade_url = https://www.engageny.org/resource/kindergarten-mathematics
Grade: Grade 1 Mathematics grade_url = https://www.engageny.org/resource/grade-1-mathematics
Grade: Grade 2 Mathematics grade_url = https://www.engageny.org/resource/grade-2-mathematics
Grade: Grade 3 Mathematics grade_url = https://www.engageny.org/resource/grade-3-mathematics
Grade: Grade 4 Mathematics grade_url = https://www.engageny.org/resource/grade-4-mathematics
Grade: Grade 5 Mathematics grade_url = https://www.engageny.org/resource/grade-5-mathematics
Grade: Grade 6 Mathematics grade_url = https://www.engageny.org/resource/grade-6-mathematics
Grade: Grade 7 Mathematics grade_url = https://www.engageny.org/resource/grade-7-mathematics
Grade: Grade 8 Mathematics grade_url = https://www.engageny.org/resource/grade-8-mathematics
Grade: Algebra I grade_url = https://www.enga

In [4]:
## Debugging web_resource_tree.json

In [5]:
def print_web_resource_tree(web_resource_tree):
    print('----')
    print('WEB RESOURCE TREE:', 'title:', web_resource_tree['title'], '  len(children) =', len(web_resource_tree['children']))
    # print('      description:', web_resource_tree['description'][0:60]+'..')
    for category in web_resource_tree['children']:
        print('   - Category   title:', category['title']) # len(category['title']))
        # print('           desciption:', category['description'][0:60]+'..') # len(category['description']))

        for resource in category['children']:
            # print(resource)
            if 'kind' not in resource:
                resource['kind'] = resource['type']
            print('      - Resource (%s):' % resource['kind'], resource['title'])
            for child in resource['children']:
                # print(child)
                print('         - Child (%s):' % child['kind'], child['title'])
    print('\n\n')

    

with open(os.path.join(TREES_DATA_DIR,CRAWLING_STAGE_OUTPUT)) as json_file:
    web_resource_tree = json.load(json_file)
    print_web_resource_tree(web_resource_tree)



----
WEB RESOURCE TREE: title: Engage NY Web Resource Tree (ELS and CCSSM)   len(children) = 0





## Calling chef method for debugging...


In [4]:
from engageny_chef import EngageNYChef
chef = EngageNYChef()

chef_args = None
chef_options = {}
chef.crawl(chef_args, chef_options)


Crawling results stored in chefdata/trees/web_resource_tree.json
